<a href="https://colab.research.google.com/github/man-is-h/face_model/blob/master/face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import dlib
import scipy.misc
import imageio
import numpy as np
import os

In [ ]:
#get face detector for dlib
face_dec = dlib.get_frontal_face_detector()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#get pose predictor
shape_pred = dlib.shape_predictor('/content/drive/My Drive/Colab Notebooks/face recognition models/shape_predictor_68_face_landmarks.dat')

#get the face recognition model
face_recognition_model = dlib.face_recognition_model_v1('/content/drive/My Drive/Colab Notebooks/face recognition models/dlib_face_recognition_resnet_model_v1.dat')

#tolerance for face comparison
TOLERANCE = 0.5

In [ ]:
#this function will give us face encodings
def get_face_encodings(image_path) :
  #load image using scipy
  image = imageio.imread(image_path)
  
  #detect using face detector
  detected_faces = face_dec(image,1)

  #get pose/landmarks of those detected faces
  shape_faces = [shape_pred(image, face) for face in detected_faces]

  #for every face detected compute the face encoding
  return [np.array(face_recognition_model.compute_face_descriptor(image, face_pose, 1)) for face_pose in shape_faces]

In [ ]:
#takes a list of known faces for comparison
def compare_face_encodings(known_faces, face):
  #finds the differences between faces using norm
  #returns true/false based on match
  #a match occurs when the norm is less than tolerance value
  return (np.linalg.norm(known_faces-face, axis=1) <= TOLERANCE)

In [ ]:
#this func return the name of person whose face matches with the given image or not found
def find_match(known_faces, names, face):
  #call compare_face_encodings to know if there is a match
  matches = compare_face_encodings(known_faces, face)

  #return name of matches
  count = 0
  for match in matches:
    if match:
      return names[count]
    count += 1

  return 'Not Found'

In [ ]:
#get face encodings for each face in image folder
#filtering on .jpg extension
image_filenames = filter(lambda x: x.endswith('jpg'), os.listdir('/content/drive/My Drive/Colab Notebooks/face recognition models/images'))
#sort in alphabetical order
image_filenames = sorted(image_filenames)

#get full paths to images
paths = ['/content/drive/My Drive/Colab Notebooks/face recognition models/images/' + x for x in image_filenames]

#list of face encodings
face_encodings = []

for path_to_image in paths:
  #get face encodings from the image
  face_encodings_in_image = get_face_encodings(path_to_image)

  #ensure only one face is there
  if(len(face_encodings_in_image)) != 1 :
    print("Please change image: " + path_to_image + " - it has " + str(len(face_encodings_in_image)) + " faces; it can only have one")
    exit()

    #append the encoding to the list of encodings
  face_encodings.append(face_encodings_in_image[0])

In [ ]:
#get path to all test images
test_filenames = filter(lambda x: x.endswith('jpg'), os.listdir('/content/drive/My Drive/Colab Notebooks/face recognition models/test'))

paths_test = ['/content/drive/My Drive/Colab Notebooks/face recognition models/test/' + x for x in test_filenames]

#list of names by eliminating .jpg extension
names = [x[:-4] for x in image_filenames]

#iterate over test_images to find match one by one
for paths_to_test in paths_test :
  face_encoding_test = get_face_encodings(paths_to_test)

  if(len(face_encoding_test)) != 1 :
    print("Please change image: " + paths_to_test + " - it has " + str(len(face_encoding_test)) + " faces; it can only have one")
    exit() 

  #find match
  match = find_match(face_encodings, names, face_encoding_test[0])

  print(paths_to_test, match)
  

/content/drive/My Drive/Colab Notebooks/face recognition models/test/2.jpg manish
/content/drive/My Drive/Colab Notebooks/face recognition models/test/1.jpg steve
/content/drive/My Drive/Colab Notebooks/face recognition models/test/3.jpg bill
/content/drive/My Drive/Colab Notebooks/face recognition models/test/4.jpg elon
/content/drive/My Drive/Colab Notebooks/face recognition models/test/6.jpg pichai
/content/drive/My Drive/Colab Notebooks/face recognition models/test/5.jpg zuckerberg
/content/drive/My Drive/Colab Notebooks/face recognition models/test/7.jpg Not Found
